<a href="https://colab.research.google.com/github/kmk4444/System_engineering/blob/main/Ragas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://docs.ragas.io/en/v0.1.21/concepts/metrics/answer_relevance.html

In [1]:
pip install langchain_openai ragas

      Successfully uninstalled langchain-0.3.20


In [3]:
import os
import pandas as pd
from langchain_openai import ChatOpenAI
from ragas import evaluate, EvaluationDataset
from ragas.metrics import AnswerRelevancy, Faithfulness, FactualCorrectness, AnswerCorrectness
from ragas.llms import LangchainLLMWrapper

# OpenAI API Anahtarını Ayarla
os.environ["OPENAI_API_KEY"] = "-proj----"

# Excel dosyasını oku (B sütununda sorular, C sütununda referans cevaplar, D sütununda deepseek-r1 cevapları)
df = pd.read_excel("/content/Data karşılaştırma.xlsx")  # Dosya yolunuza göre güncelleyiniz

# Kolonları uygun değişkenlere alalım
questions = df.iloc[:, 1].tolist()
reference_answers = df.iloc[:, 2].tolist()
deepseek_answers = df.iloc[:, 3].tolist()

# LLM Modelini Başlat
llm = ChatOpenAI(model="gpt-4o")  # GPT-4o değerlendirme için kullanılacak
evaluator_llm = LangchainLLMWrapper(llm)

# Değerlendirme için veri setini oluştur
dataset = []
for question, deepseek_response, reference in zip(questions, deepseek_answers, reference_answers):
    dataset.append(
        {
            "user_input": question,
            "retrieved_contexts": [],  # Bağlam kullanmadığımız için boş liste
            "response": deepseek_response,
            "reference": reference
        }
    )

# Dataset'i Ragas ile formatla
evaluation_dataset = EvaluationDataset.from_list(dataset)

# Ragas değerlendirmesini gerçekleştir
result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
        AnswerRelevancy(),
        Faithfulness(),
        FactualCorrectness(),
        AnswerCorrectness()
    ],
    llm=evaluator_llm
)

# Sonuçları yazdır
print("\n--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---\n")
print(result)


Evaluating:   0%|          | 0/200 [00:00<?, ?it/s]


--- DeepSeek-r1 Değerlendirme Sonuçları (ChatGPT-4o Referans Olarak Kullanıldı) ---

{'answer_relevancy': 0.8581, 'faithfulness': 0.0339, 'factual_correctness(mode=f1)': 0.2758, 'answer_correctness': 0.3510}


In [4]:
# İlk 3 elemanı yazdırma
print("Questions - İlk 3 Satır:", questions[:3])
print("Reference Answers - İlk 3 Satır:", reference_answers[:3])
print("DeepSeek Answers - İlk 3 Satır:", deepseek_answers[:3])

# Son 3 elemanı yazdırma
print("\nQuestions - Son 3 Satır:", questions[-3:])
print("Reference Answers - Son 3 Satır:", reference_answers[-3:])
print("DeepSeek Answers - Son 3 Satır:", deepseek_answers[-3:])


Questions - İlk 3 Satır: ['Sistem mühendisliği nedir ve temel amacı nedir?', 'V-Model (V-Şeması) nedir ve hangi aşamaları içerir?', '“Sistem gereksinimleri” ile “kullanıcı gereksinimleri” arasındaki fark nedir?']
Reference Answers - İlk 3 Satır: ['Sistem mühendisliği, karmaşık sistemlerin tasarım, geliştirme, entegrasyon ve yaşam döngüsü yönetimiyle ilgilenen disiplinler arası bir alanıdır. Temel amacı, sistemleri en verimli şekilde tasarlamak, sürekliliğini sağlamak ve uçtan uca yönetmektir.', 'V-Model, sistem veya yazılım geliştirme yaşam döngüsünü tanımlamak için kullanılan bir modeldir. Sol kolunda gereksinim analizi ve tasarım faaliyetleri, sağ kolunda ise test ve doğrulama faaliyetleri bulunur. Ortada entegrasyon aşaması yer alır ve süreç, gereksinimlerden doğrulamaya kadar sistematik bir biçimde ilerler.', 'Sistem gereksinimleri, bir sistemin teknik ve işlevsel niteliklerini tanımlar ve tasarım ekibi tarafından anlaşılması gereken ayrıntıları içerir. Kullanıcı gereksinimleri ise